In [7]:
import cv2
from transformers import pipeline
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

def capture_image():
    # Open a connection to the webcam
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Could not open webcam")
        return None

    # Capture a single frame
    ret, frame = cap.read()

    if not ret:
        print("Failed to capture image")
        return None

    # Release the webcam
    cap.release()

    # Save the captured image
    image_path = "captured_image.jpg"
    cv2.imwrite(image_path, frame)

    return image_path

def recognize_emotion(image_path):
    # Load a pre-trained emotion recognition model
    emotion_recognizer = pipeline('image-classification', model='google/vit-base-patch16-224-in21k')

    # Recognize the emotion
    emotion = emotion_recognizer(image_path)

    return emotion[0]['label']  # Return the detected emotion label

def get_song_recommendations(emotion):
    # Spotify API credentials
    client_id = "380a6b3535dc420a905dccf328a0e165"  # Replace with your actual client_id
    client_secret = "b0ca7947448246d28aafc40f49610cf4"  # Replace with your actual client_secret

    # Authenticate with Spotify
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

    # Map emotion to mood (you can customize this)
    mood_map = {
        "Happy": "happy",
        "Sad": "sad",
        "Angry": "angry",
        "Disgusted": "disgust",
        "Fearful": "fear",
        "Neutral": "neutral",
        "Surprised": "surprise"
    }

    mood = mood_map.get(emotion, "venom")  # Default to "happy" if emotion is not mapped

    # Search for tracks based on mood
    results = sp.search(q=mood, type='track', limit=5)

    recommendations = []
    for idx, track in enumerate(results['tracks']['items']):
        recommendations.append({
            "name": track['name'],
            "artist": track['artists'][0]['name'],
            "url": track['external_urls']['spotify']
        })

    return recommendations

def main():
    # Capture an image from the camera
    image_path = capture_image()

    if image_path:
        # Recognize the emotion in the captured image
        emotion = recognize_emotion(image_path)
        print(f"Detected Emotion: {emotion}")

        # Get song recommendations based on the detected emotion
        recommendations = get_song_recommendations(emotion)
        print("Recommended Songs:")
        for rec in recommendations:
            print(f"{rec['name']} by {rec['artist']} - {rec['url']}")

if __name__ == "__main__":
    main()


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Detected Emotion: LABEL_1
Recommended Songs:
Venom - Music From The Motion Picture by Eminem - https://open.spotify.com/track/3kpYJjvM8Ja6btr5hEJLWc
Venom - Music From The Motion Picture by Eminem - https://open.spotify.com/track/2SL6oP2YAEQbqsrkOzRGO4
Venom by $uicideboy$ - https://open.spotify.com/track/4RYhSQFFlquCTb8XRMJgFU
Venom - Music From The Motion Picture by Eminem - https://open.spotify.com/track/4lcN22k03kmMXMxI26GYPF
Venom by Ghostemane - https://open.spotify.com/track/7tUXZgLbOCvRJ9FBlairoU
